In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

# Релиз 2.0

In [2]:
path = 'https://raw.githubusercontent.com/AlexeyKamaev/test/main/_data.csv'
df = pd.read_csv(path)
pd.set_option('display.max_columns', None)

In [3]:
df = df.rename(columns={'Площадь, м2':'Площадь'}).drop('Ссылка на объявление',axis = 1).drop('Unnamed: 0',axis = 1).drop('Телефоны',axis = 1)
df = df[df['Адрес'].str.contains('Москва')]
df = df.dropna(subset=['Метро'])

In [4]:
# Обработка колонки Цена и получение из нее цены и залога
def get_price(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  return int(r.split()[0])

def get_deposit(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  try:
      if int(r.split()[2]) < 1000 or int(r.split()[2]) > int(r.split()[0]) * 6:
        return int(r.split()[0])
      else:
        return int(r.split()[2])
  except:
      return int(r.split()[0])
    
x1 = df['Цена'].map(get_price) 
x2 = df['Цена'].map(get_deposit) 

In [5]:
df.insert(9, "Цена за месяц",x1)
df.insert(10,'Залог',x2)

In [6]:
# Обработка колонки Площадь, м2 и получение из нее площади 
def first_element(x):
    y = re.findall(r'\d+', x)
    return y[0]

df['Этаж'] = df['Дом'].map(first_element).astype('int')
def second_element(x):
    y = re.findall(r'\d+', x)
    return y[1]

df['Этажность дома'] = df['Дом'].apply(second_element).astype('int')
from operator import methodcaller
def func(x):
  return float(x[0])
point_place = df['Площадь'].map(methodcaller("split", "/")).map(func).copy()
df.insert(5, "Общая площадь",point_place)

In [7]:
# Получение колонки Станция из Метро
def func (y):
    y[:y.find('(')]
    return y[:y.find('(')].strip()

def func2 (y):
    y[:y.find('(')]
    return y[y.find('м. ')+3:].strip()

x = df['Метро'].map(func).map(func2).copy()
df.insert(3, "Станция", x)

In [8]:
# Удаление ненужных колонок
df.drop(['Метро', 'Цена','Описание','Дополнительно','Название ЖК'], axis= 1, inplace=True)
df = df[['ID  объявления','Адрес','Станция','Серия дома', 'Общая площадь',	'Площадь', 'Этаж',	'Этажность дома', 'Высота потолков, м', 'Площадь комнат, м2', 'Количество комнат','Ремонт','Балкон', 'Окна','Санузел','Лифт','Мусоропровод', 'Тип','Дом',	'Парковка',	'Цена за месяц','Залог','Можно с детьми/животными']].copy()
df = df.rename(columns={'ID  объявления':'ID'}).reset_index(drop=True)

In [9]:
# Удаление колонок, которые были рассмотрены как не влияющие на стоимость квартир
df.drop(['Лифт', 'Санузел', 'Высота потолков, м', 'Балкон', 'Парковка', 'Мусоропровод', 'Окна'], axis= 1, inplace=True)

In [11]:
# Добавление данных об окргуах и районах для группировки по соответствующим округам станциям 
metro = pd.read_csv('metro - metro.csv', sep=',').rename(columns={'Station':'Станция'})
def func(s):
    s = s.replace('ё','е')
    s = s.replace('  ',' ')
    if s == 'Марк':
        s = 'Алтуфьево'
    elif s == 'Перерва':
        s = 'Братиславская'
    elif s.lower() == 'библиотека им. ленина':
        s = 'библиотека имени ленина'
    elif s.lower() == 'бескудниково':
        s = 'селигерская'
    elif s.lower() == 'гражданская':
        s = 'дмитровская'
    return s.lower().strip()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19391 entries, 0 to 19390
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        19391 non-null  int64  
 1   Адрес                     19391 non-null  object 
 2   Станция                   19391 non-null  object 
 3   Серия дома                2088 non-null   object 
 4   Общая площадь             19391 non-null  float64
 5   Площадь                   19391 non-null  object 
 6   Этаж                      19391 non-null  int32  
 7   Этажность дома            19391 non-null  int32  
 8   Площадь комнат, м2        12306 non-null  object 
 9   Количество комнат         18868 non-null  object 
 10  Ремонт                    16944 non-null  object 
 11  Тип                       19391 non-null  object 
 12  Дом                       19391 non-null  object 
 13  Цена за месяц             19391 non-null  int64  
 14  Залог 

In [12]:
# Получение координат станций метро

metro = metro.drop_duplicates('Станция').loc[:,['Станция','coor']]
metro.loc[:,'Станция'] = metro.loc[:,'Станция'].map(func)
df.loc[:,'Станция'] = df.loc[:,'Станция'].map(func)

m = df.merge(metro, how = 'left', on='Станция').dropna(how='all')
df.insert(3, "metro_coord",m['coor'])

In [13]:
# Получение координат квартир
house = pd.read_csv('coordinates.csv', sep=',')
house = house.rename(columns={'ID  объявления':'ID'}).reset_index(drop=True).drop('Unnamed: 0', axis=1)
me = df.merge(house, how = 'left', on='ID').dropna(how='all')

In [14]:
df.insert(2, "house_coord",me['coor'])
df.insert(5,'house_metro',0)
df.insert(6,'house_kremlin',0)

# Релиз 2.0 Расстояния
1. Расстояние до Кремля от дома: house_kremlin
2. Расстояние от дома до метро: house_metro
3. Если нет координаты дома, то среднее расстоние до метро в районе // ср расстония от района до кремля
4. Если нет координат метро, то макс расстояние до метро + макс расстояние до креля

In [16]:
# Функция, вычисляющая расстояние до Кремля
import geopy
import geopy.distance
import numpy as np
kremlin = (55.7516212, 37.618122044334896)
def leng(coord_1, coord_2=kremlin):
    try:
        temp = coord_1.split(',')
        coord_1 = tuple((float(temp[0][1:]),float(temp[1][:-1])))
        if coord_2 == kremlin:
            return round(geopy.distance.geodesic(coord_1, coord_2).km,3)
        else:
            temp = coord_2.split(',')
            coord_2 = tuple((float(temp[0][1:]),float(temp[1][:-1])))
            return round(geopy.distance.geodesic(coord_1, coord_2).km,3)
    except Exception as e:
        return np.NaN

In [17]:
# Создание колонки расстояние от дома до кремля
df['house_kremlin'] = df['house_coord'].map(leng)

In [18]:
# Создание колонки расстояние от дома до метро
df['house_metro'] = [leng(a,b) for a, b in df.loc[:,['house_coord','metro_coord']].itertuples(index=False)]

In [19]:
# Создание колонки расстояние от метро (района) до кремля
df.insert(6,'metro_kremlin',0)
df['metro_kremlin'] = df['metro_coord'].map(leng)

In [20]:
# Удаление ненужных колонок
df.drop(['house_coord', 'metro_coord'], axis= 1, inplace=True)

# Релиз 2.0 Заполнение пропусков

In [22]:
# Обработка колонки Площадь комнат

def process_area(x):
    if pd.notna(x):
        numbers = [float(num) for num in re.findall(r'\d+\.\d+|\d+', x)]
        return numbers if numbers else [float(x)]
    else:
        return 0

df['Площадь комнат, м2'] = df['Площадь комнат, м2'].apply(process_area)


In [24]:
# Обработка колонки Количество комнат
df['Количество комнат'] = df['Количество комнат'].str.extract('(\d+)')


In [51]:
# Заполнение пропусков модой = 2
def func(x):
    try:
        return int(x)
    except:
        return np.NaN
df['rooms'] = df['Количество комнат'].map(func)
df['rooms'].fillna(2, inplace=True)

In [ ]:
# Удаление ненужных колонок 
df.drop(['Серия дома', 'Площадь', 'Площадь комнат, м2', 'Тип'], axis=1, inplace=True)


In [55]:
# Обработка колонки Можно с детьми/животными и разбиение ее на 2: children, animal. Кодирование этих колонок
def process_children(x):
    try:
        if 'детьми' in x:
            return 1
        else:
            return 0
    except:
        return 0
df['Children'] = df['Можно с детьми/животными'].map(process_children)

def process_animal(x):
    try:
        if 'животными' in x:
            return 1
        else:
            return 0
    except:
        return 0

df['Animal'] = df['Можно с детьми/животными'].map(process_animal)

In [56]:
df.drop('Можно с детьми/животными', axis=1, inplace=True)

In [58]:
# Обработка колонки Ремонт и разбиение на отдельные колонки. Кодирование этих колонок
def f1(x):
    try:
        if 'Евро' in x:
            return 1
        else:
            return 0
    except:
        return 0
def f2(x):
    try:
        if 'Дизайн' in x:
            return 1
        else:
            return 0
    except:
        return 0
def f3(x):
    try:
        if 'Косм' in x:
            return 1
        else:
            return 0
    except:
        return 0
def f4(x):
    try:
        if 'Без' in x:
            return 1
        else:
            return 0
    except:
        return 0

In [59]:
df['euro'] = df['Ремонт'].map(f1)
df['design'] = df['Ремонт'].map(f2)
df['cosmetic'] = df['Ремонт'].map(f3)
df['no_rem'] = df['Ремонт'].map(f4)

In [60]:
# Удаление ненужных колонок
df.drop('Ремонт', axis=1, inplace=True)
df.drop('Количество комнат', axis=1, inplace=True)

In [80]:
# Обработка колонки Дом и получение из нее Типа дома. Заполнение пропусков
df['type_house'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 5 else 'Неизвестно' for row in df['Дом']] 
filter = (df['type_house'].isin(('старый фонд', 'Щитовой', 'Деревянный')))
df = df[~filter]


# Релиз 2.0 Районы

In [16]:
df = pd.read_csv('data.csv').drop('Unnamed: 0',axis=1).rename(columns={'Адрес':'adress','Станция':'Station','Этаж':'floor','Этажность дома':'height','Цена за месяц':'price','Залог':'deposit' })

,ID,adress,Station,house_metro,metro_kremlin,house_kremlin,Общая площадь,floor,height,Дом,price,deposit,rooms,Children,Animal,euro,design,cosmetic,no_rem,type_house
0,271271157,"Москва, улица Новый Арбат, 27",смоленская,0.390,2.224,2.410,200.0,5,16,"5/16, Монолитный",500000,500000,4.0,1,1,0,1,0,0,Монолитный
1,271634126,"Москва, улица Новый Арбат, 27",смоленская,0.390,2.224,2.410,198.0,5,16,"5/16, Монолитно-кирпичный",500000,500000,4.0,1,0,0,1,0,0,Монолитно-кирпичный
2,271173086,"Москва, улица Новый Арбат, 27",смоленская,0.390,2.224,2.410,200.0,5,16,5/16,500000,500000,4.0,1,0,1,0,0,0,Неизвестно
3,272197456,"Москва, переулок Плотников, 21С1",смоленская,NaN,2.224,NaN,170.0,5,6,5/6,400000,400000,4.0,0,1,1,0,0,0,Неизвестно
4,273614615,"Москва, улица Новый Арбат, 15",арбатская,1.469,0.593,2.059,58.0,12,26,"12/26, Панельный",225000,225000,2.0,0,0,1,0,0,0,Панельный
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19313,215565511,"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",говорово,NaN,18.647,NaN,35.0,10,14,10/14,42000,42000,2.0,0,0,1,0,0,0,Неизвестно
19314,274654844,"Москва, Производственная улица, 8к1, ш. Боровс...",солнцево,NaN,16.330,NaN,38.7,5,18,"5/18, Монолитный",45000,45000,1.0,0,0,1,0,0,0,Монолитный
19315,268679909,"Москва, Боровский проезд, 11",солнцево,NaN,16.330,NaN,43.1,5,5,"5/5, Кирпичный",50000,50000,2.0,1,0,0,1,0,0,Кирпичный
19316,274807525,"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",солнцево,NaN,16.330,NaN,52.5,8,23,"8/23, Монолитный",55000,50000,2.0,0,0,1,0,0,0,Монолитный


In [10]:
dist = pd.read_csv('districts.csv', sep = ';').loc[1:,:]

In [50]:
# 
def f(s):
    s = s.strip()
    s = s.lower()
    return s
dist['Station']=dist['Station'].map(f)
dist = dist.loc[:,['Station','Line','AdmArea','District']].drop_duplicates('Station')
df = df.merge(dist, on='Station',how='left').dropna(how='all')

In [55]:
# Переименование колонок
df = df.drop(['adress','Дом'],axis=1).rename(columns={'Station':'station','Общая площадь':'sq','Children':'child','Animal':'animal','Line':'line','AdmArea':'area','District':'dist'})

In [64]:
# Удаление пропусков по колонкам Площадь и Расстояние до Кремля, поскольку эти признаки являются клбчевыми для предсказания цены
df = df.dropna(subset=['area','metro_kremlin'])

In [83]:
data = df.house_metro
data.sort_values(ascending=False)

filter = (df['house_metro'] > 20)
# df = df[~filter]

In [97]:
def func(a,b):
    if np.isnan(b):
        return a
    return b

x = df.copy()

x['house_kremlin'] =  [func(a,b) for a, b in df.loc[:,['metro_kremlin','house_kremlin']].itertuples(index=False)]
df = x